In [1]:
import jiwer
import xml.etree.ElementTree as ET
import os
import glob
import re

In [2]:
import re

def normalizeText(s):
    """
    Ritorna testo minuscolo, senza punteggiatura, senza numeri e senza sequenze di apice seguite da spazio.
    :param: s (string)
    :return: string
    """
    s = s.encode('utf-8', 'ignore').decode('utf-8')
    
    s = s.lower()
    
    # Rimuovo tutta la punteggiatura (tranne ') sostituendola con SPAZIO
    chars = '[%s]+' % re.escape('!$&()"*+,-./:;<=>?@[\\]^_`{|}~\n\t')
    s = re.sub(chars, ' ', s)
    
    # Rimuovo tutti i numeri
    s = re.sub(r'\d+', ' ', s)
    
    # Aggiungo uno spazio dopo l'apice
    s = re.sub(r"(?<=\')", " ", s)  # Usa lookbehind per aggiungere uno spazio dopo l'apice
    
    # Rimuovo doppi spazi
    s = re.sub(r"(\s)+", " ", s)
    
    # Rimuovo spazi iniziali e finali
    s = s.strip()
    
    return s


In [22]:
def calcola_wer(riferimenti, predetti):
    """
    Calcola WER (non nel modo standard), se vuoi quella std somma (I+D+S)/N
    """
    # Legge il testo del file di predizione
    with open(predetti, "r", encoding="utf-8") as file:
        testo = normalizeText(file.read())
    
    # Legge il testo del file di riferimento
    with open(riferimenti, "r", encoding="utf-8") as ref_file:
        testo_riferimento = normalizeText(ref_file.read())
    
    # Calcola la Word Error Rate (WER) e ottiene i dettagli
    dettagli = jiwer.compute_measures(testo_riferimento, testo)
    
    # Calcola il numero di parole nel testo di riferimento
    num_parole = len(testo_riferimento.split())
    
    inserimenti = dettagli['insertions']
    cancellazioni = dettagli['deletions']
    sostituzioni = dettagli['substitutions']
    wer = (inserimenti + cancellazioni + sostituzioni) / num_parole
    
    return wer, inserimenti, cancellazioni, sostituzioni, num_parole


In [6]:
def estrai_testo_xml(input_filename):
    """
    Estrai trascrizioni da XML
    """
    with open(input_filename, "r", encoding="utf-8") as file:
        xml_content = file.read()

    try:
        # Parsing XML
        root = ET.fromstring(xml_content)
        print("XML caricato con successo.")
    except ET.ParseError:
        print(input_filename)
    except Exception as e:
        print(input_filename)

    parole = [token.get("data-text") for token in root.findall(".//span[@class='token pvText-container']")]

    frase = " ".join(parole)

    output_filename = os.path.splitext(input_filename)[0] + "_output.txt"

    with open(output_filename, "w", encoding="utf-8") as output_file:
        output_file.write(frase)
        
    return output_filename

In [31]:
lang = "ENG"

In [32]:
remove = True
if remove:
    outfiles = []
    directory_paths = [f"/workspace/tirocinio/fine-tuning/TestSets/{lang}/ACS", f"/workspace/tirocinio/fine-tuning/TestSets/{lang}/AW_E2E", f"/workspace/tirocinio/fine-tuning/TestSets/{lang}/GCS"]
    for dir in directory_paths:
        files = glob.glob(os.path.join(dir, "*_output.txt"))
        for file in files:
            os.remove(file)

In [ ]:
outfiles = []
directory_paths = [f"/workspace/tirocinio/fine-tuning/TestSets/{lang}/ACS", f"/workspace/tirocinio/fine-tuning/TestSets/{lang}/AW_E2E", f"/workspace/tirocinio/fine-tuning/TestSets/{lang}/GCS"]
for dir in directory_paths:
    files = glob.glob(os.path.join(dir, "*.txt"))
    for file in files:
        outfile = estrai_testo_xml(file)
        outfiles.append(outfile)

In [ ]:
for dir in directory_paths:
    # Variabili per sommare i valori di WER e contare i file
    somma_wer = 0
    somma_inserimenti = 0
    somma_cancellazioni = 0
    somma_sostituzioni = 0
    num_files = 0
    parole_totali = 0
    
    for file_path in outfiles:
        ref = os.path.join(f"/workspace/tirocinio/fine-tuning/TestSets/{lang}/Ref", os.path.basename(file_path).replace("_output.txt", ".ref"))
        if file_path.startswith(dir):
            wer_value, inserimenti, cancellazioni, sostituzioni, num_parole = calcola_wer(riferimenti=ref, predetti=file_path)
            somma_wer += wer_value
            somma_inserimenti += inserimenti
            somma_cancellazioni += cancellazioni
            somma_sostituzioni += sostituzioni
            parole_totali += num_parole
            num_files += 1

    if num_files > 0:
        #wer_media = somma_wer / num_files
        wer_media = somma_wer / num_files
        print(f"Directory: {dir}")
        print(f"  WER media: {wer_media:.4f}")
        print(f"  Totale Inserimenti: {somma_inserimenti}")
        print(f"  Totale Cancellazioni: {somma_cancellazioni}")
        print(f"  Totale Sostituzioni: {somma_sostituzioni}")
        print(f"  Totale Parole: {parole_totali}")
        print()
    else:
        print(f"Nessun file trovato in {dir}.")

# WER standard

### WER ENG
Directory: /workspace/tirocinio/fine-tuning/TestSets/ENG/ACS    
  WER media: 0.0924  
  Totale Inserimenti: 444  
  Totale Cancellazioni: 181  
  Totale Sostituzioni: 706  
  Totale Parole: 13999  

Directory: /workspace/tirocinio/fine-tuning/TestSets/ENG/AW_E2E   
  WER media: 0.0756  
  Totale Inserimenti: 196  
  Totale Cancellazioni: 194  
  Totale Sostituzioni: 718  
  Totale Parole: 13999  

Directory: /workspace/tirocinio/fine-tuning/TestSets/ENG/GCS  
  WER media: 0.2573  
  Totale Inserimenti: 358  
  Totale Cancellazioni: 1661  
  Totale Sostituzioni: 1668  
  Totale Parole: 13999  


### WER ITA
 
Directory: /workspace/tirocinio/fine-tuning/TestSets/ITA/ACS  
  WER media: 0.0928  
  Totale Inserimenti: 753  
  Totale Cancellazioni: 306  
  Totale Sostituzioni: 601  
  Totale Parole: 18309  

Directory: /workspace/tirocinio/fine-tuning/TestSets/ITA/AW_E2E  
  WER media: 0.1129  
  Totale Inserimenti: 291  
  Totale Cancellazioni: 895  
  Totale Sostituzioni: 779  
  Totale Parole: 18309  
  
Directory: /workspace/tirocinio/fine-tuning/TestSets/ITA/GCS  
  WER media: 0.1802  
  Totale Inserimenti: 274   
  Totale Cancellazioni: 1508  
  Totale Sostituzioni: 1505  
  Totale Parole: 18309  


# Trascrittore NVIDIA
Trascrivo i file forniti (prima li ho tagliati in modo che siano <6 minuti)

In [68]:
lang = "ITA"

In [69]:
import os
import glob
import shutil
from pydub import AudioSegment

In [84]:
def split_audio(file_path, segment_duration=4):
    audio = AudioSegment.from_wav(file_path)
    
    # Ottieni la durata in millisecondi
    duration_ms = len(audio)
    duration_min = duration_ms / (1000 * 60)
    print(f"Durata totale: {duration_min:.2f} minuti")

    # Calcola il numero di segmenti
    segment_duration_ms = segment_duration * 60 * 1000  # 4 minuti in millisecondi
    num_segments = (duration_ms // segment_duration_ms) + (1 if duration_ms % segment_duration_ms != 0 else 0)

    output_folder = f"/workspace/tirocinio/fine-tuning/TestSets/{lang}/short_audio"

    base_name = os.path.splitext(os.path.basename(file_path))[0]

    for i in range(num_segments):
        start_time = i * segment_duration_ms
        end_time = start_time + segment_duration_ms
        if end_time > duration_ms:
            end_time = duration_ms
        
        # Estrai il segmento
        segment = audio[start_time:end_time]
        
        # Salva il segmento
        segment.export(os.path.join(output_folder, f"{base_name}_{i + 1}.wav"), format="wav")
        print(f"Segmento {i + 1} salvato.")

In [ ]:
# Divido i file audio
dir = f"/workspace/tirocinio/fine-tuning/TestSets/{lang}/Audio"
for nome_file in os.listdir(dir):
        if nome_file.endswith(".wav"):
            split_audio(os.path.join(dir,nome_file))

In [ ]:
import nemo.collections.asr as nemo_asr
asr_model = nemo_asr.models.EncDecHybridRNNTCTCBPEModel.from_pretrained(model_name="stt_multilingual_fastconformer_hybrid_large_pc")

In [ ]:
# Trascrivo i file
audiodir = f"TestSets/{lang}/short_audio"
transcript_dir = f"/workspace/tirocinio/fine-tuning/TestSets/{lang}/NVIDIA"
files = glob.glob(os.path.join(audiodir, "*.wav"))

i = 0
for file in files:
    transcript_file = os.path.join(transcript_dir, os.path.basename(file).replace(".wav", ".txt"))
    print(f"Trascrivo il file: {file}")
    transcript = asr_model.transcribe([file])
    with open(transcript_file, "w", encoding='utf-8') as outfile:
        testo = transcript[0][0]
        outfile.write(testo)

    dest = f"TestSets/{lang}/short_audio/Done/{os.path.basename(file)}"

    if os.path.exists(dest):
        os.remove(dest)

    shutil.move(file, dest)
    i += 1

In [87]:
from collections import defaultdict

# Unisce i file trascritti con nome comune dando il path della cartella
def join_predictions(dir):
    """
    Unisce i file trascritti con nome comune
    """
    gruppi_file = defaultdict(list)
    predictions = []

    # Prendo tutti i file trascritti e li raggruppo per il prefisso del nome senza _n
    files = glob.glob(os.path.join(dir, '*.txt'))
    for file in files:
        nome_file = os.path.basename(file)
        nome_base = "_".join(nome_file.split("_")[:-1])
        gruppi_file[nome_base].append(nome_file)

    # Per ogni gruppo di file con lo stesso prefisso
    for nome_base, file_list in gruppi_file.items():
        file_list.sort()
        
        # Apro il nuovo file che conterrà tutte le trascrizioni di quel gruppo
        nome_output = os.path.join(dir, f"{nome_base}.txt")
        with open(nome_output, 'w', encoding='utf-8') as file_output:
            for nome_file in file_list:
                percorso_file = os.path.join(dir, nome_file)
                
                # Leggo il contenuto di ogni file del gruppo e lo scrivo
                with open(percorso_file, 'r', encoding='utf-8') as file_input:
                    contenuto = file_input.read()
                    file_output.write(contenuto + " ")
                
                # Una volta trascritto il file lo sposto nella cartella {dir}/Done
                destination_path = os.path.join(dir, "Done", nome_file)
                if os.path.exists(destination_path):
                    os.remove(destination_path)

                shutil.move(percorso_file, destination_path)
        predictions.append(nome_output)
    
    return predictions

In [88]:
def pair_files(predictions, references):
    """
    Accoppia ogni prediction alla propria reference
    """
    # Crea dei dizionari per mappare il nome del file senza estensione con il percorso completo
    pred_dict = {os.path.splitext(os.path.basename(p))[0]: p for p in predictions}
    ref_dict = {os.path.splitext(os.path.basename(r))[0]: r for r in references}

    # Trova le coppie con lo stesso nome e crea una lista di tuple
    paired_files = [(pred_dict[name], ref_dict[name]) for name in pred_dict if name in ref_dict]

    return paired_files

In [89]:
# Definisco variabili
dir = f"/workspace/tirocinio/fine-tuning/TestSets/{lang}/NVIDIA"

In [90]:
# Elimino i file output.txt (non serve sempre)
for nome_file in os.listdir(dir):
    if nome_file.endswith("output.txt"):
        os.remove(os.path.join(dir, nome_file))

In [91]:
# Join delle trascrizioni e muovo i file in transcripted
outfiles = join_predictions(dir)

In [92]:
ref = glob.glob(os.path.join(f"/workspace/tirocinio/fine-tuning/TestSets/{lang}/Ref", '*.ref'))   
pairs = pair_files(predictions=outfiles, references=ref)

In [ ]:
print("Pres:")
for p in outfiles:
    print(f"\t{p}")

print("ref")
for p in ref:
    print(f"\t{p}")

print("Pairs")
for p in pairs:
    print(f"\t{p}")

In [ ]:
# Variabili per sommare i valori di WER e contare i file
somma_wer = 0
somma_inserimenti = 0
somma_cancellazioni = 0
somma_sostituzioni = 0
num_files = 0
parole_totali = 0

for pred, reff  in pairs:
    wer_value, inserimenti, cancellazioni, sostituzioni, num_parole = calcola_wer(riferimenti=reff, predetti=pred)
    print(f"{pred}: {wer_value}")
    somma_wer += wer_value
    somma_inserimenti += inserimenti
    somma_cancellazioni += cancellazioni
    somma_sostituzioni += sostituzioni
    parole_totali += num_parole
    num_files += 1

if num_files > 0:
    wer_media = somma_wer / num_files
    print(f"Directory: {dir}")
    print(f"  WER media: {wer_media:.4f}")
    print(f"  Totale Inserimenti: {somma_inserimenti}")
    print(f"  Totale Cancellazioni: {somma_cancellazioni}")
    print(f"  Totale Sostituzioni: {somma_sostituzioni}")
    print(f"  Totale Parole: {parole_totali}")
    print()
else:
    print(f"Nessun file trovato in {dir}.")

# WER standard ENG
Directory: /workspace/tirocinio/fine-tuning/TestSets/ENG/NVIDIA  
  WER media: 0.1045  
  Totale Inserimenti: 452  
  Totale Cancellazioni: 313  
  Totale Sostituzioni: 1037  
  Totale Parole: 16706  

# WER standard ITA
Directory: /workspace/tirocinio/fine-tuning/TestSets/ITA/NVIDIA  
  WER media: 0.1994  
  Totale Inserimenti: 330  
  Totale Cancellazioni: 1676  
  Totale Sostituzioni: 1338  
  Totale Parole: 18309  